In [1]:
import pandas as pd

In [2]:
# 处理广告数据
ad_feature_file = pd.read_csv('../data/ad_feature.csv')
print(ad_feature_file.head(12))
ad_feature_file.info()
ad_feature_file.shape

    adgroup_id  cate_id  campaign_id  customer     brand   price
0        63133     6406        83237         1   95471.0  170.00
1       313401     6406        83237         1   87331.0  199.00
2       248909      392        83237         1   32233.0   38.00
3       208458      392        83237         1  174374.0  139.00
4       110847     7211       135256         2  145952.0   32.99
5       607788     6261       387991         6  207800.0  199.00
6       375706     4520       387991         6       NaN   99.00
7        11115     7213       139747         9  186847.0   33.00
8        24484     7207       139744         9  186847.0   19.00
9        28589     5953       395195        13       NaN  428.00
10       23236     5953       395195        13       NaN  368.00
11      300556     5953       395195        13       NaN  639.00
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 846811 entries, 0 to 846810
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype  
--

(846811, 6)

In [13]:
for i in ad_feature_file.columns:
    null_rate = ad_feature_file[i].isna().sum() / len(ad_feature_file) * 100
    if null_rate > 0:
        print("{} null rate: {} %".format(i, round(null_rate, 2)))

# 处理缺失值
ad_feature_file.dropna(inplace=True)  # 删除含有缺失值的行
# 处理重复值
ad_feature_file.drop_duplicates(inplace=True)  # 删除重复行
#将ad_feature_file中的brand列转为整型
ad_feature_file['brand'] = ad_feature_file['brand'].astype(int)
#保存文件
ad_feature_file.to_csv('ad_feature_data.csv', index=False)

NameError: name 'ad_feature_file' is not defined

In [ ]:
# 处理用户行为数据
behavior_file = pd.read_csv('../data/behavior_log.csv',nrows=500000)
print(behavior_file.head(12))
behavior_file.info()
behavior_file.shape

In [ ]:
for i in behavior_file.columns:
    null_rate = behavior_file[i].isna().sum() / len(behavior_file) * 100
    if null_rate > 0:
        print("{} null rate: {} %".format(i, round(null_rate, 2)))
behavior_file.rename(columns={'user':'userid'},inplace=True)
behavior_file.rename(columns={'cate':'cate_id'},inplace=True)

In [ ]:
behavior_file['time_stamp']=pd.to_datetime(behavior_file['time_stamp'],unit='s')
# 从转换后的数据中分别提取：日期、时间、小时，组成新的列
behavior_file['date'] = behavior_file['time_stamp'].dt.date
behavior_file['time'] = behavior_file['time_stamp'].dt.time
behavior_file['hour'] = behavior_file['time_stamp'].dt.hour
behavior_file = behavior_file.drop(['time_stamp'],axis=1)
behavior_file.to_csv('behavior_data.csv',index=False,sep=',')

In [3]:
# 用户基本信息
user_profile_file = pd.read_csv('../data/user_profile.csv')
print(user_profile_file.head(12))
user_profile_file.info()

    userid  cms_segid  cms_group_id  final_gender_code  age_level  \
0      234          0             5                  2          5   
1      523          5             2                  2          2   
2      612          0             8                  1          2   
3     1670          0             4                  2          4   
4     2545          0            10                  1          4   
5     3644         49             6                  2          6   
6     5777         44             5                  2          5   
7     6211          0             9                  1          3   
8     6355          2             1                  2          1   
9     6823         43             5                  2          5   
10    6972          5             2                  2          2   
11    9293          0             5                  2          5   

    pvalue_level  shopping_level  occupation  new_user_class_level   
0            NaN               3

In [4]:
from sklearn.impute import SimpleImputer
for i in user_profile_file.columns:
    null_rate = user_profile_file[i].isna().sum() / len(user_profile_file) * 100
    if null_rate > 0:
        print("{} null rate: {} %".format(i, round(null_rate, 2)))
city_level = user_profile_file.loc[:,'new_user_class_level '].values.reshape(-1,1)  # loc：对索引名进行切片   
si = SimpleImputer(strategy = 'most_frequent')  # 实例化，使用众数填补
user_profile_file.loc[:,'new_user_class_level '] = si.fit_transform(city_level) # fit_transform一步训练导出结果
user_profile_file.info()
# 利用KNN对消费档次进行预测填充
from sklearn.neighbors import KNeighborsClassifier
columns = ['userid', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
       'shopping_level', 'occupation', 'new_user_class_level ','pvalue_level']
user_data = user_profile_file[columns]
# pavlue_level为空的样本作为测试集
pvalue_null = user_data.loc[user_data['pvalue_level'].isnull().values == True]
# pavlue_level不空的样本作为训练集
pvalue_nonull = user_data.loc[user_data['pvalue_level'].isnull().values == False]
X_train_user,y_train_user = pvalue_nonull.iloc[:,:-1],pvalue_nonull.iloc[:,-1]
X_test_user,y_test_user = pvalue_null.iloc[:,:-1],pvalue_null.iloc[:,-1]
knn = KNeighborsClassifier(n_neighbors=3,weights='distance')
knn.fit(X_train_user,y_train_user)
# 得到预测结果
y_test_user = knn.predict(X_test_user)
y_test_user = pd.DataFrame(y_test_user)
y_test_user.columns = ['pvalue_level']
X_test_user.reset_index(drop=True,inplace=True)
pvalue_null = pd.concat([X_test_user,y_test_user],axis=1)
user = pd.concat([pvalue_nonull,pvalue_null],ignore_index=False)
# 将处理过后的user信息保存
user.to_csv('user_data.csv',index=False,sep=',')

pvalue_level null rate: 54.24 %
new_user_class_level  null rate: 32.49 %
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1061768 entries, 0 to 1061767
Data columns (total 9 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   userid                 1061768 non-null  int64  
 1   cms_segid              1061768 non-null  int64  
 2   cms_group_id           1061768 non-null  int64  
 3   final_gender_code      1061768 non-null  int64  
 4   age_level              1061768 non-null  int64  
 5   pvalue_level           485851 non-null   float64
 6   shopping_level         1061768 non-null  int64  
 7   occupation             1061768 non-null  int64  
 8   new_user_class_level   1061768 non-null  float64
dtypes: float64(2), int64(7)
memory usage: 72.9 MB


In [6]:
# 样本
raw_sample_file = pd.read_csv('../data/raw_sample.csv', nrows=500000)
print(raw_sample_file.head(12))
raw_sample_file.info()

      user  time_stamp  adgroup_id          pid  nonclk  clk
0   581738  1494137644           1  430548_1007       1    0
1   449818  1494638778           3  430548_1007       1    0
2   914836  1494650879           4  430548_1007       1    0
3   914836  1494651029           5  430548_1007       1    0
4   399907  1494302958           8  430548_1007       1    0
5   628137  1494524935           9  430548_1007       1    0
6   298139  1494462593           9  430539_1007       1    0
7   775475  1494561036           9  430548_1007       1    0
8   555266  1494307136          11  430539_1007       1    0
9   117840  1494036743          11  430548_1007       1    0
10  739815  1494115387          11  430539_1007       1    0
11  623911  1494625301          11  430548_1007       1    0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 6 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   user      

In [7]:
for i in raw_sample_file.columns:
    null_rate = raw_sample_file[i].isna().sum() / len(raw_sample_file) * 100
    if null_rate > 0:
        print("{} null rate: {} %".format(i, round(null_rate, 2)))
# 将处理过后的user信息保存
raw_sample_file.to_csv('sample_data.csv',index=False)

In [38]:
print(f'dataset表的维度：{raw_sample_file.shape}')
print(f'user表的维度：{user.shape}')
print(f'ads表的维度：{ad_feature_file.shape}')

# 修改表dataset中列名user为userid，以便后面基于主键连接
raw_sample_file.rename(columns={'user':'userid'},inplace=True)
raw_sample_file.head()

# 将数据集dataset与用户基本信息表user合并，基于主键userid，how='right'表示以右边表为基准连接
user_dataset = pd.merge(user,raw_sample_file,on='userid',how='right')
print(f'user_dataset表的维度：{user_dataset.shape}')

# 将数据集与广告基本信息表ads合并，基于主键adgroup_id，how='right'表示以右边表为基准连接
ads_user_dataset = pd.merge(ad_feature_file,user_dataset,on='adgroup_id',how='right')
print(f'ads_user_dataset表的维度：{ads_user_dataset.shape}')

ads_user_dataset.to_csv('data.csv',index=False,sep=',')

dataset表的维度：(500000, 6)
user表的维度：(1061768, 9)
ads表的维度：(600481, 6)
user_dataset表的维度：(500000, 14)
ads_user_dataset表的维度：(500000, 19)


In [8]:
data = pd.read_csv('data.csv')
# 获取data每列的缺失值占比
data_null = data.isnull().sum()/len(raw_sample_file)*100
data_null = data_null.drop(data_null[data_null==0].index).sort_values(ascending=False)     # 将缺失值占比从高到低排序
missing_data = pd.DataFrame({'Missing Ratio(%)':data_null})
print(f'dataset含有缺失值的属性个数：{len(data_null)}')
print(missing_data)
data.dropna(axis=0, how='any',inplace=True)

dataset含有缺失值的属性个数：13
                       Missing Ratio(%)
cate_id                         15.6260
campaign_id                     15.6260
customer                        15.6260
brand                           15.6260
price                           15.6260
cms_segid                        5.8584
cms_group_id                     5.8584
final_gender_code                5.8584
age_level                        5.8584
shopping_level                   5.8584
occupation                       5.8584
new_user_class_level             5.8584
pvalue_level                     5.8584


In [40]:
# 将数据中的时间戳形式转换为日期和时间形式
import datetime
import time
data['time_stamp']=pd.to_datetime(data['time_stamp'],unit='s')
data['time_stamp']
# 从转换后的数据中分别提取：日期、时间、小时，组成新的列
data['date'] = data['time_stamp'].dt.date
data['time'] = data['time_stamp'].dt.time
data['hour'] = data['time_stamp'].dt.hour
# 调整数据集data的列顺序:将'data'、'time'、'hour'这三列数据调至'time_stamp'列后
columns = ['adgroup_id', 'cate_id', 'campaign_id', 'customer', 'brand', 'price',
       'userid', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level',
       'shopping_level', 'occupation', 'new_user_class_level ', 'pvalue_level',
       'time_stamp', 'date', 'time', 'hour', 'pid', 'nonclk', 'clk']
data = data[columns]
data = data.drop(['time_stamp','time','nonclk'],axis=1)
data.to_csv('user_ad_data.csv',index=False,sep=',')